# Test Chisel notebook
> An example that defines a module over multiple notebook cells.

We must first import our Chisel dependencies:
```Scala
import $file.^.source.load_ivy
```

In [1]:
//export
//default_exp test
import $file.^.source.load_ivy

import $file.$                

In [2]:
//export
import chisel3.{Input => Input}
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.tester.RawTester.{test => Test}

import chisel3.{Input => Input}

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3.tester.RawTester.{test => Test}

### define the IO
> If we define our IO as a standalone bundle we are able to reuse it for other modules

In [3]:
//export
class ALUIO(width: Int) extends Bundle {
    val a = Input(SInt(width.W))
    val b = Input(SInt(width.W))
    val out = Output(SInt())

    override def cloneType = new ALUIO(width).asInstanceOf[this.type]
}

defined class ALUIO

### A skeleton module will be parent class of all of our methods
> This module currently has only IO defined. We will use Scala implicits to inject functionality into it.

In [4]:
//export 
class ALUSkeleton(width: Int) extends Module {
    val io = IO(new ALUIO(width))
}

defined class ALUSkeleton

### The add method from Add class will be injected into the ALUSkeleton class
> This syntax will be used to inject the ```add``` method into our skeleton module:
```Scala
class Add(m: ALUSkeleton) {
    def add(a: SInt, b: SInt): SInt = a +& b
}
```

In [5]:
//export 
class Add(m: ALUSkeleton) {
    def add(a: SInt, b: SInt): SInt = a +& b
}

defined class Add

Similarly we can define a `sub` method that will be added

In [6]:
//export 
class Sub(m: ALUSkeleton) {
    def sub(a: SInt, b: SInt): SInt = a -& b
}
println("changed auto-gen file")

changed auto-gen file


defined class Sub

### Here multiple methods will be injected
> Both `mul` and `div` will be injected.

In [7]:
//export 
class MulDiv(m: ALUSkeleton) {
    println("changed auto-gen file")
    def mul(a: SInt, b: SInt): SInt = a * b
    def div(a: SInt, b: SInt): SInt = a / b
}

defined class MulDiv

### Adding the methods to our skeleton module:
> implicit def call will add the methods of the RHS classes to the implicit def parameter: ALUSkeleton class.
Note the name ```includeAdd``` does not matter as it won't be called again, but is good practice to give it a clear name of what it is doing (i.e includAdd <=> include the Add class' methods)

In [8]:
//export 
implicit def includeAdd(m: ALUSkeleton) = new Add(m)
implicit def includeSub(m: ALUSkeleton) = new Sub(m)
implicit def includeMulDiv(m: ALUSkeleton) = new MulDiv(m)

defined function includeAdd
defined function includeSub
defined function includeMulDiv

### Use our implicitly defined functions
We extend our ALUSkeleton giving us access to all of our implicitly defined methods. We can match based on parameterized operator (at compile time) and then only instantiate hardware for that given operation. 

In [11]:
//export
/** This Operator module performs 1 type of operation depending on 'op' parameter */
class Operator(op: String, width: Int) extends ALUSkeleton(width) {
    op match {
        // Call on the implicit function
        case "+" => io.out := this.add(io.a, io.b)
        case "-" => io.out := this.sub(io.a, io.b)
        case "*" => io.out := this.mul(io.a, io.b)
        case "/" => io.out := this.div(io.a, io.b)
        case _ => io.out := 0.S
    }
}

defined class Operator

### Summary
> While this includes a few more steps than simply implementing everything directly into an Operator module, it can actually save some headaches when building much larger moudules. The best case for developing this way is if you're defining a specific arithmetic implmentation, which this example was meant to outline. For example a multiplier may require many sub methods to implement, and these implementation details can be hidden from the module using it. Also working like this puts you in the mindset of pulling out independent pieces of logic to write functions for that can later be reused in other modules. 

In [12]:
//hide
val width = 10
Test(new Operator(op="+", width)) { c =>
    c.io.a.poke(5.S); c.io.b.poke(3.S)
    c.io.out.expect(8.S)
}


Test(new Operator(op="-", width)) { c =>
    c.io.a.poke(5.S); c.io.b.poke(3.S)
    c.io.out.expect(2.S)
}


Test(new Operator(op="*", width)) { c =>
    c.io.a.poke(5.S); c.io.b.poke(3.S)
    c.io.out.expect(15.S)
}


Test(new Operator(op="/", width)) { c =>
    c.io.a.poke(5.S); c.io.b.poke(3.S)
    c.io.out.expect(1.S)
}


Test(new Operator(op="%", width)) { c =>
    c.io.a.poke(5.S); c.io.b.poke(3.S)
    c.io.out.expect(0.S)
}

[info] [0.001] Elaborating design...
[info] [0.164] Done elaborating.
file loaded in 0.033898958 seconds, 6 symbols, 2 statements
test Operator Success: 0 tests passed in 2 cycles in 0.056397 seconds 35.46 Hz
[info] [0.002] Elaborating design...
[info] [0.066] Done elaborating.
file loaded in 0.023028458 seconds, 6 symbols, 2 statements
test Operator Success: 0 tests passed in 2 cycles in 0.043912 seconds 45.55 Hz
[info] [0.001] Elaborating design...
changed auto-gen file
[info] [0.066] Done elaborating.
file loaded in 0.007488167 seconds, 6 symbols, 2 statements
test Operator Success: 0 tests passed in 2 cycles in 0.009698 seconds 206.23 Hz
[info] [0.005] Elaborating design...
changed auto-gen file
[info] [0.058] Done elaborating.
file loaded in 0.02027775 seconds, 6 symbols, 2 statements
test Operator Success: 0 tests passed in 2 cycles in 0.007809 seconds 256.12 Hz
[info] [0.001] Elaborating design...
[info] [0.069] Done elaborating.
file loaded in 0.002808916 seconds, 6 symbols, 2 

width: Int = 10

In [13]:
//export
class OperatorTester(c: Operator, op: String, width: Int) extends PeekPokeTester(c) {
  // 100 random tests
  val cycles = 10
  import scala.util.Random
  import scala.math.min
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(1 << (width - 1))
    val in_b = Random.nextInt(1 << (width - 1))
    poke(c.io.a, in_a)
    poke(c.io.b, in_b)
    val exp = op match {
        case "+" => in_a + in_b
        case "-" => in_a - in_b
        case "*" => in_a * in_b
        case "/" => in_a / in_b
    }
//     println(s"$in_a $op $in_b = $exp")
    expect(c.io.out, exp)
    }
  }

defined class Tester

In [14]:
//hide
for (op <- Seq("+", "-", "*", "/")) {
    assert(Driver(() => new Operator(op, width)) {c => new OperatorTester(c, op, width)})
}
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.048] Done elaborating.
Total FIRRTL Compile Time: 73.5 ms
Total FIRRTL Compile Time: 47.2 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1612850250836
[info] [0.005] 93 + 437 = 530
[info] [0.007] 373 + 347 = 720
[info] [0.008] 288 + 436 = 724
[info] [0.008] 266 + 332 = 598
[info] [0.009] 331 + 79 = 410
[info] [0.010] 460 + 209 = 669
[info] [0.010] 411 + 493 = 904
[info] [0.011] 283 + 468 = 751
[info] [0.011] 26 + 289 = 315
[info] [0.014] 44 + 256 = 300
test Operator Success: 10 tests passed in 5 cycles taking 0.022560 seconds
[info] [0.016] RAN 0 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.012] Done elaborating.
Total FIRRTL Compile Time: 47.1 ms
Total FIRRTL Compile Time: 21.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1612850251712
[info] [0.001] 331 - 101 = 230
[info] [0.002] 288 - 180 = 108
[info] [0.003] 97 - 475 = -378
[info] [0.005] 418 - 369 = 49
[info] [0.006] 99 - 2

### Some test funcs/classes/objects to be exported from this notebook:

In [ ]:
//export NewScript
def FuncToNewScript(): Unit = {print("hello world")}

In [ ]:
//export NewScript2
def FuncToNewScript2(): Unit = {print("hello squirrel")}

In [ ]:
//export
object TestObj {}

In [ ]:
//export
abstract class LotsOfParams(a: Int, b: List[(Int, Int)], c: BigInt)